In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import datetime as dt
%matplotlib inline
sns.set_context('notebook')

path = '/'.join(os.getcwd().split('/')[:-2])
print(path)

tci = pd.read_csv(path+'/data/model_data/tci_1_0.csv', parse_dates=['Date'], dtype={'PIN': str})
cols = ['parcel','vacant','Date','Survey Category','Survey Grade','SPA_NAME']
tci = tci[cols]
ppns = set(tci['parcel'])
dates = dict(zip(tci.parcel, tci.Date))

print(tci.shape)

/Volumes/Dropbox/largetransfer/luc/carter
(113132, 6)


## Violations

In [2]:
vs = pd.read_csv(path+'/data/clean_data/violations.csv', parse_dates = [1,4])
vs = vs.loc[vs.apply(lambda x: x.v_file_date < dates[x.parcel], axis=1)]

In [3]:
vs_copy = vs.groupby('VIN').last().reset_index()
# vs_copy = vs_copy[['parcel','v_file_date']].groupby('parcel').count().reset_index()
vs_copy['vs_count'] = vs_copy.apply(lambda x: \
        len(vs_copy[(vs_copy.parcel == x.parcel) & (vs_copy.v_file_date > dates[x.parcel]-pd.DateOffset(years=1))]), axis=1)
vs_copy = vs_copy.groupby('parcel').last()
vs_copy = vs_copy.rename(columns={'vs_count':'v_total_1yr'})
df = pd.merge(tci, vs_copy[['v_total_1yr']], \
           how='left', left_on='parcel', right_index=True)


In [5]:
vs_copy = vs.groupby('VIN').last().reset_index()
# vs_copy = vs_copy[['parcel','v_file_date']].groupby('parcel').count().reset_index()
vs_copy['vs_count'] = vs_copy.apply(lambda x: \
        len(vs_copy[(vs_copy.parcel == x.parcel) & (vs_copy.v_file_date > dates[x.parcel]-pd.DateOffset(years=1))]), axis=1)
vs_copy = vs_copy.groupby('parcel').last()
vs_copy = vs_copy.rename(columns={'vs_count':'v_total_1yr'})
df = pd.merge(tci, vs_copy[['v_total_1yr']], \
           how='left', left_on='parcel', right_index=True)

vs_copy = vs.groupby('VIN').last().reset_index()
# vs_copy = vs_copy[['parcel','v_file_date']].groupby('parcel').count().reset_index()
vs_copy['vs_count'] = vs_copy.apply(lambda x: \
        len(vs_copy[(vs_copy.parcel == x.parcel) & (vs_copy.v_file_date > dates[x.parcel]-pd.DateOffset(years=2))]), axis=1)
vs_copy = vs_copy.groupby('parcel').last()
vs_copy = vs_copy.rename(columns={'vs_count':'v_total_2yr'})
df = pd.merge(df, vs_copy[['v_total_2yr']], \
           how='left', left_on='parcel', right_index=True)

df.loc[df.v_total_1yr.isnull(), 'v_total_1yr'] = 0
df.loc[df.v_total_2yr.isnull(), 'v_total_2yr'] = 0

## Complaints

In [6]:
cp = pd.read_csv(path+'/data/clean_data/complaints.csv', parse_dates = [2],index_col=[0], dtype=str)
cp = cp.loc[cp.apply(lambda x: x.c_file_date < dates[x.parcel], axis=1)]
cp = cp.sort_values('c_file_date')

In [7]:
cp_copy = cp[(cp.c_file_date > dt.datetime(2006,1,1))]

# get parcels that have had a closed-board up request during the time period
df.loc[df.parcel.isin(cp_copy.loc[(cp_copy.c_wf_task_status=='Closed-Board Up Requested'),'parcel']),'boarded'] = 1
df.loc[df.boarded.isnull(),'boarded'] = 0

In [8]:
cp_copy = cp_copy.groupby('complaint_id').last().reset_index()

df = pd.merge(df, cp_copy.loc[(cp_copy.c_type_of_complaint=='O.V.V.')\
                ,['c_file_date','parcel']]\
               .groupby('parcel')\
               .last(), how='left', left_on='parcel', right_index=True)
df = df.rename(columns = {'c_file_date':'days_since_ovv'})
df.loc[df.days_since_ovv.isnull(),'days_since_ovv'] = dt.datetime(2006,1,1)
df['days_since_ovv'] = df.apply(lambda x: (dates[x.parcel]-x.days_since_ovv).days, axis=1)

# c2 = pd.merge(cp_copy.loc[(cp_copy.c_type_of_complaint=='O.V.V.'),\
#                               ['c_file_date','parcel']].groupby('parcel').last(), tci[['parcel','Date']], how='left',\
#         left_index=True, right_on='parcel').set_index('parcel')
# c2['days_since_ovv'] = c2.Date.apply(lambda x: (x-dt.datetime(2006,1,1)))


c2 = pd.merge(cp_copy.loc[(cp_copy.c_type_of_complaint=='O.V.V.'),\
                              ['c_file_date','parcel']].groupby('parcel').count(), tci[['parcel','Date']], how='left',\
        left_index=True, right_on='parcel').set_index('parcel')
c2['days'] = c2.Date.apply(lambda x: (x-dt.datetime(2006,1,1)).days/365.0)
c2['c_ovv_num'] = c2['c_file_date']/c2['days']

# average number of OVV complaints per year since 2006
df = pd.merge(df, c2[['c_ovv_num']]
                    ,how='left', left_on='parcel', right_index=True)
df.loc[df.c_ovv_num.isnull(),'c_ovv_num'] = 0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113132 entries, 0 to 113131
Data columns (total 11 columns):
parcel             113132 non-null object
vacant             113132 non-null int64
Date               113132 non-null datetime64[ns]
Survey Category    113132 non-null object
Survey Grade       113132 non-null object
SPA_NAME           113132 non-null object
v_total_1yr        113132 non-null float64
v_total_2yr        113132 non-null float64
boarded            113132 non-null float64
days_since_ovv     113132 non-null int64
c_ovv_num          113132 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(2), object(4)
memory usage: 10.4+ MB


In [10]:
df.to_csv(path+'/data/model_data/tci_2_2_model.csv', index=False)